In [ ]:
def mapper(line):
  n1,n2 =line.split(',')
  return (n1[1:-1],n2[1:-1])

In [ ]:
def mapper2(line):
  n1,n2 =line.split(',')
  if n1 != n2:
    if n1<n2:
      return (n1[1:-1],n2[1:-1])
    else:
      return (n2[1:-1],n1[1:-1])

# Simplificación del grafo

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext # SparkContext hace que sea interactivo
import sys

sc = SparkContext("local", "App Name")

In [ ]:
# grafos de ejemplo
!gdown  1Y3HsyP8rhkxBNrB_qnaF7ERe-IjIQHip
!gdown  13M-ZNH52d6eKlVEU10GeQsIYxumi1mMe
!gdown  11BR6kyHbwRv91B1TdnbKfnbl8kpTEWRi
!gdown  1hutIxFRSDVnQ0RBZ3ffZFsKDBfWfUtd2
!gdown  1_J2_SxV2LOTOSouTqFZlGUe8-gJul8Vr

Downloading...
From: https://drive.google.com/uc?id=1Y3HsyP8rhkxBNrB_qnaF7ERe-IjIQHip
To: /content/g0.txt
100% 24.0/24.0 [00:00<00:00, 86.0kB/s]
Downloading...
From: https://drive.google.com/uc?id=13M-ZNH52d6eKlVEU10GeQsIYxumi1mMe
To: /content/g1.txt
100% 56.0/56.0 [00:00<00:00, 245kB/s]
Downloading...
From: https://drive.google.com/uc?id=11BR6kyHbwRv91B1TdnbKfnbl8kpTEWRi
To: /content/g3.txt
100% 48.0/48.0 [00:00<00:00, 209kB/s]
Downloading...
From: https://drive.google.com/uc?id=1hutIxFRSDVnQ0RBZ3ffZFsKDBfWfUtd2
To: /content/g4.txt
100% 72.0/72.0 [00:00<00:00, 324kB/s]
Downloading...
From: https://drive.google.com/uc?id=1_J2_SxV2LOTOSouTqFZlGUe8-gJul8Vr
To: /content/g10.txt
100% 88.0/88.0 [00:00<00:00, 319kB/s]


In [ ]:
sc.textFile('g3.txt').map(mapper2).collect() # sc.textFile('g3.txt') nos da una rdd, el map nos da una transformacion y luego le aplico una acción (mirar ppt)
#collect en la realidad no lo vamos a usar en todo caso un take, aplicamos un

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'F'), None, ('A', 'F')]

In [ ]:
sc.textFile('g3.txt').map(mapper2).distinct().collect() #distinct me quita todos los pares repetidos

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'F'), None]

In [ ]:
# con un filter hacemos q nos quite los none
sc.textFile('g3.txt').map(mapper2).filter(lambda x: x!=None).distinct().collect()
# todo esto se va ejecutando, y spark de manera interna decide en q orden ejecuta esto. todo esto son locales, por ejemplo el mapeo tambien el filtro te digo si lo pasa o no lo pasas. EL distinct si que
# para hacer el distinct hay q pasar por la fase del shuffer, entonces esta funcion distinct es wide y es compleja necesita el shuffle.
# si pongo el filter detras del distinct,
# una transformacion puede ser narrow o si necesita trabajar con el dataset es wide

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'F')]

otras alternativas:

In [ ]:
def mapper3(line):
    n1,n2 =line.split(',')
    if n1 != n2:
      if n1<n2:
        return (n1[1:-1],n2[1:-1]),None
      else:
       return (n2[1:-1],n1[1:-1]),None

In [ ]:
sc.textFile('g3.txt').map(mapper3).\
  filter(lambda x: x!=None).\
  groupByKey().keys().collect()

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'F')]

In [ ]:
# si queremos ver los pasos intermedios
sc.textFile('g3.txt').map(mapper3).\
  filter(lambda x: x!=None).\
  groupByKey().mapValues(list).collect() #quito las keys

[(('A', 'B'), [None]),
 (('A', 'C'), [None]),
 (('A', 'D'), [None]),
 (('A', 'F'), [None, None])]

otra:

In [ ]:
def salida(x):
  result=[]
  for n in set(x[1]):
    result.append((x[0]),n)
  return result


  # el mapper 2 me da los grafos ya ordenados


In [ ]:
sc.textFile('g4.txt').map(mapper2).\
  filter(lambda x: x!=None).groupByKey().\
  flatMap(salida).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10) (3c52338a20ea executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwar

codigo de fernando para simplificar el grafo:
.... vuelve a hacer un mapeo recibe una tupla y te devuelve una tupla, un lmabda donde

lo ordena y cuando lo tiene ordenado, hace un reducekey y un append de listas . el reducebykey hace lo mismo q el groupbykey del profe, sabe q tiene cadena de caracteres y

In [ ]:
sc.textFile('g3.txt').map(lambda x: x.split(',')).collect()
# por ver como va quedando el codigo

In [ ]:
sc.textFile('g3.txt').map(lambda x: x.split(',')).map(lambda x: (x[0],x[1]) if x[0]==x[1] if x[0]!=x[1] if x[0]).collect()

# clase **lunes 28** de octubre de 2024

In [ ]:
sc.textFile('access')